# LIBRARY

In [1]:
from __future__ import division
from nltk.tokenize import word_tokenize, RegexpTokenizer
from collections import defaultdict
import pandas as pd
import requests
import sqlite3
from sqlite3 import Error
import os
import re
import sys, time
import nltk
import numpy as np
#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Connecting to Database

In [2]:
#connecting to database
conn = sqlite3.connect("DBNews.db")
dataset = pd.read_sql_query("SELECT * FROM t_comments", conn)
cursor = conn.cursor()

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
dataset

,comment_id,comment,baseline,correct_comment,comment_ptr
0,4207096,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA",-1,None,1
1,4207096,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�,1,None,2
2,4207096,Salut buat Gus Miftah,1,None,3
3,4207096,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat,1,None,4
4,4207096,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..,1,None,5
...,...,...,...,...,...
598092,4416660,212 nggak pernah ngadain acara beginian ya....terus program mereka dalam keagamaan apa ya....kalo mereka ngadain nya abis saya nulis ini...berarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong,None,None,598093
598093,4416660,TBG... Sayank.. salah pilih kendraaan..�,None,None,598094
598094,4416660,Saya bersama TGB pilih Jokowi�,None,None,598095
598095,4416660,"Hijrah dari hoax ke non hoax, dari ngawur ke bener, dari saling menjelekkan ke saling puji dan hormat!",None,None,598096


In [5]:
#ambil yg kolom comment aja
dataset = dataset[['comment']]

In [6]:
dataset

,comment
0,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA"
1,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�
2,Salut buat Gus Miftah
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat
4,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..
...,...
598092,212 nggak pernah ngadain acara beginian ya....terus program mereka dalam keagamaan apa ya....kalo mereka ngadain nya abis saya nulis ini...berarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong
598093,TBG... Sayank.. salah pilih kendraaan..�
598094,Saya bersama TGB pilih Jokowi�
598095,"Hijrah dari hoax ke non hoax, dari ngawur ke bener, dari saling menjelekkan ke saling puji dan hormat!"


## Read data from database

In [7]:
dataset

,comment
0,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA"
1,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�
2,Salut buat Gus Miftah
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat
4,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..
...,...
598092,212 nggak pernah ngadain acara beginian ya....terus program mereka dalam keagamaan apa ya....kalo mereka ngadain nya abis saya nulis ini...berarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong
598093,TBG... Sayank.. salah pilih kendraaan..�
598094,Saya bersama TGB pilih Jokowi�
598095,"Hijrah dari hoax ke non hoax, dari ngawur ke bener, dari saling menjelekkan ke saling puji dan hormat!"


In [8]:
#removing symbol
def strip_symbol(text):
    return re.sub(r'[^\w\s]',' ',text)

#removing integer
def strip_integer(text):
    return re.sub(r'[\d+]', ' ', text)

#removing regex and tokenizing
def regex_tokenizer(text):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

def make_it_lowercase(text):
    return text.lower()

In [9]:
dataset['comment'] = dataset['comment'].apply(lambda text:strip_symbol(text))
dataset['comment'] = dataset['comment'].apply(lambda text:strip_integer(text))
dataset['comment'] = dataset['comment'].apply(lambda text:make_it_lowercase(text))
dataset['comment'] = dataset['comment'].apply(lambda text:regex_tokenizer(text))

C:\Users\RuiHan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\RuiHan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\RuiHan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [10]:
dataset

,comment
0,"[wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]"
1,"[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]"
2,"[salut, buat, gus, miftah]"
3,"[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, smua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]"
4,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...
598092,"[nggak, pernah, ngadain, acara, beginian, ya, terus, program, mereka, dalam, keagamaan, apa, ya, kalo, mereka, ngadain, nya, abis, saya, nulis, ini, berarti, mereka, sebenernya, cuman, gerakan, politik, yg, cari, pembenaran, dan, kekuasaan, doang, dong]"
598093,"[tbg, sayank, salah, pilih, kendraaan]"
598094,"[saya, bersama, tgb, pilih, jokowi]"
598095,"[hijrah, dari, hoax, ke, non, hoax, dari, ngawur, ke, bener, dari, saling, menjelekkan, ke, saling, puji, dan, hormat]"


In [11]:
df=dataset[:2000]
df['comment']

0       [wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]                                                                                                                                               
1       [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                
2       [salut, buat, gus, miftah]                                                                                                                                                                                                  
3       [yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, smua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]
4       [ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]              

In [12]:
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# df['stem_comment'] = df['comment'].apply(lambda x: [stemmer.stem(y) for y in x])

In [13]:
# df['stem_comment']

In [14]:
# df['stem_comment'].to_csv('stemmed_data.csv', index=False)
# dff = pd.read_csv('stemmed_data.csv', names =['stem_comment'])
# dff

In [15]:
singkatan_kamus_alay = pd.read_csv("colloquial-indonesian-lexicon.csv")
singkatan_kamus_alay = singkatan_kamus_alay[singkatan_kamus_alay.columns[0:2]]
# singkatan_kamus_alay

In [16]:
singkatan_dataset = pd.read_csv("kumpulan_singkatan_dan_kata_dasar.csv", names=['singkatan', 'kata asli'])
# singkatan_dataset

In [17]:
dataset['comment']

0         [wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]                                                                                                                                                                                
1         [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                                                 
2         [salut, buat, gus, miftah]                                                                                                                                                                                                                                   
3         [yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, smua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, 

In [18]:
#import tqdm
#utk show progress
from tqdm import tqdm

In [19]:
for i in tqdm(range(len(df['comment']))):
    for j in range(len(df['comment'][i])):
        try:
            words = df['comment'][i][j]
            df['comment'][i][j] = list(singkatan_kamus_alay['formal'])[(list(singkatan_kamus_alay['slang']).
                                                                            index(df['comment'][i][j]))]
            ynormalized = df['comment'][i][j]
            print(df['comment'][i][j])
        except:
            pass
            print(df['comment'][i][j])

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

wkwkwk
enggak
sekalian
pak
selawat
ke
dpr
kan
banyak
koruptor
disana


  0%|                                                                                 | 1/2000 [00:00<03:27,  9.62it/s]

mantab
ini
baru
dakwah
yang
sejati
kalau
ada
tempat
umum
mah
deh
biasa
ini
luar
biasa


  0%|                                                                                 | 2/2000 [00:00<03:55,  8.47it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████▉| 1997/2000 [04:31<00:00,  6.64it/s]

enak
ya
bisa
pakai
jasa
kpk
basi
pak
pencitraan
pakai
kpk
dulu
menterinujung
nya
main
comot
saja
yang
sudah
jadi
tersangka
kpk
kok
malah
menjabat
hadeh
suram
jok
kampanyemu


100%|█████████████████████████████████████████████████████████████████████████████▉| 1999/2000 [04:31<00:00,  7.15it/s]

blasssss
enggak
ada
wibawanya


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [04:31<00:00,  7.36it/s]


In [20]:
for i in tqdm(range(len(df['comment']))):
    for j in range(len(df['comment'][i])):
        try:
            words = df['comment'][i][j]
            df['comment'][i][j] = list(singkatan_dataset['kata asli'])[(list(singkatan_dataset['singkatan']).
                                                                            index(df['comment'][i][j]))]
            ynormalized = df['comment'][i][j]
            print(df['comment'][i][j])
        except:
            pass
            print(df['comment'][i][j])

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

wkwk
enggak
sekalian
pak
selawat
ke
dapur
kan
banyak
koruptor
disana
mantab
ini
baru
dakwah
yang
sejati
kalau
ada
tempat
umum
mah
deh
biasa
ini
luar
biasa
salut
buat
gus
miftah
yang
nyinyir
enggak
pernah
lihat
dan
baca
atau
mengetahui
sejarah
dakwah
walisongo
sunan
kali
jaga
semua
ada
cara
demi
kebaikan
asal
jangan
ajak
dengan
cara
berzina
lakukan
dengan
porsi
dan
sikon
yang
tepat


  0%|▏                                                                                | 4/2000 [00:00<01:16, 25.98it/s]

ada
adabnya
sholawat
lebih
baik
ajak
ke
majelis
ilmu
setiap
ulama
punya
jalan
dan
cara
dakwah
masing
jika
kamu
melihat
kemunkaran
lawan
dengan
tanganmu
kekuatanmu
jika
tidak
mampu
lawan
dengan
mulutmu
jika
tidak
mampu
juga
dengan
hatimu
itulah
selemah
lemahnya
iman
itu
sudah
menjadi
kewajiban
para
ulama
kalo
yang
komen
masih
ada
yang
nyinyir
ulama
berarti
otaknya
sudah
disisipin
politik


  0%|▏                                                                                | 6/2000 [00:00<01:43, 19.29it/s]

...

pns
nya
boleh
juga
dong
biar
tambah
kacau
ini
negara
cerdas
kpla
daerah
kebanyakan
pro
pemerintah
tambah
nyungsep
saja
sebelah
ciyeee
yang
ada
kepentingan
politik
kekuasaan
hhaaaaa
boleh
saja
orang
menteri
saja
nyalonin
diri
jadi
caleg
siap
tgp


 99%|█████████████████████████████████████████████████████████████████████████████▍| 1986/2000 [01:34<00:00, 32.54it/s]

mantapppp
pak
jokowi
dua
periode
oh
jadi
kpk
sekarang
milik
jokowi
seharusnya
kpk
itu
lembaga
yang
netral
mas
prabowo
juga
pakai
sama
gue
baca
komen
ki
ageng
mentaram
enggak
mutu
terus
tapi
sok
pintar
ketemu
gue
tabok
juga
ini
orang


100%|█████████████████████████████████████████████████████████████████████████████▌| 1990/2000 [01:34<00:00, 33.19it/s]

ki
ageng
mataram
pikir
dia


In [21]:
#dari sini keliatan yg udah diproses cuma dataset ke-2000, 2001 blm diperbaiki
dataset[0:2000]

,comment
0,"[wkwk, enggak, sekalian, pak, selawat, ke, dapur, kan, banyak, koruptor, disana]"
1,"[mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]"
2,"[salut, buat, gus, miftah]"
3,"[yang, nyinyir, enggak, pernah, lihat, dan, baca, atau, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, semua, ada, cara, demi, kebaikan, asal, jangan, ajak, dengan, cara, berzina, lakukan, dengan, porsi, dan, sikon, yang, tepat]"
4,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...
1995,"[pasti, kpk, memilih, machmud, md, orang, yang, bersih, rekam, jejak, nya, bagus, dan, bukan, dari, kalangan, partai]"
1996,"[hahaha, ingat, dulu, waktu, memilih, mentri, libatin, kpk, juga, tapi, rekomendasi, kpk, enggak, dipakai]"
1997,"[enak, ya, bisa, pakai, jasa, kpk]"
1998,"[basi, pak, pencitraan, pakai, kpk, dulu, menterinujung, nya, main, comot, saja, yang, sudah, jadi, tersangka, kpk, kok, malah, menjabat, hadeh, suram, jok, kampanyemu]"


In [22]:
pip install -U gensim

Requirement already up-to-date: gensim in c:\users\ruihan\anaconda3\lib\site-packages (3.8.3)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import io
import time
from datetime import timedelta
import gensim

In [24]:
#Mengolah corpus yang ada
import io
import time
from datetime import timedelta
import gensim
start_time = time.time()
print('Streaming wiki...')
id_wiki = gensim.corpora.WikiCorpus('idwiki-20200301-pages-articles-multistream.xml.bz2', lemmatize=False, dictionary={})
article_count = 0
with io.open('idwiki.txt', 'w', encoding='utf-8') as wiki_txt:
    for text in id_wiki.get_texts():
        wiki_txt.write(" ".join(text) + '\n')
        article_count += 1
        if article_count % 10000 == 0:
            print('{} articles processed'.format(article_count))
    print('total: {} articles'.format(article_count))
finish_time = time.time()
print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

Streaming wiki...


C:\Users\RuiHan\Anaconda3\lib\site-packages\gensim\utils.py:1268: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)


10000 articles processed
20000 articles processed
30000 articles processed
40000 articles processed
50000 articles processed
60000 articles processed
70000 articles processed
80000 articles processed
90000 articles processed
100000 articles processed
110000 articles processed
120000 articles processed
130000 articles processed
140000 articles processed
150000 articles processed
160000 articles processed
170000 articles processed
180000 articles processed
190000 articles processed
200000 articles processed
210000 articles processed
220000 articles processed
230000 articles processed
240000 articles processed
250000 articles processed
260000 articles processed
270000 articles processed
280000 articles processed
290000 articles processed
300000 articles processed
310000 articles processed
320000 articles processed
330000 articles processed
340000 articles processed
350000 articles processed
360000 articles processed
370000 articles processed
380000 articles processed
total: 389441 article

In [25]:
#Simpan Model
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki.txt"
namaFileOutput = "wiki_word2vec_model"
 
model = Word2Vec(LineSentence(namaFileInput), size=200, window=3, min_count=3, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.save(namaFileOutput)

2020-06-01 18:17:12,172 : INFO : running C:\Users\RuiHan\Anaconda3\lib\site-packages\ipykernel_launcher.py -f C:\Users\RuiHan\AppData\Roaming\jupyter\runtime\kernel-6a9a8063-51e2-431e-9f20-3046c8d61e7f.json
2020-06-01 18:17:12,175 : INFO : collecting all words and their counts
2020-06-01 18:17:12,178 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-01 18:17:16,873 : INFO : PROGRESS: at sentence #10000, processed 8892918 words, keeping 276391 word types
2020-06-01 18:17:19,927 : INFO : PROGRESS: at sentence #20000, processed 14228418 words, keeping 382113 word types
2020-06-01 18:17:22,076 : INFO : PROGRESS: at sentence #30000, processed 18580233 words, keeping 464527 word types
2020-06-01 18:17:24,478 : INFO : PROGRESS: at sentence #40000, processed 22543234 words, keeping 532463 word types
2020-06-01 18:17:26,727 : INFO : PROGRESS: at sentence #50000, processed 26520102 words, keeping 616513 word types
2020-06-01 18:17:28,585 : INFO : PROGRESS: at sen

2020-06-01 18:18:54,684 : INFO : EPOCH 1 - PROGRESS: at 4.17% examples, 471170 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:18:55,708 : INFO : EPOCH 1 - PROGRESS: at 4.45% examples, 473286 words/s, in_qsize 6, out_qsize 1
2020-06-01 18:18:56,741 : INFO : EPOCH 1 - PROGRESS: at 4.76% examples, 474813 words/s, in_qsize 7, out_qsize 1
2020-06-01 18:18:57,741 : INFO : EPOCH 1 - PROGRESS: at 5.07% examples, 477234 words/s, in_qsize 3, out_qsize 0
2020-06-01 18:18:58,749 : INFO : EPOCH 1 - PROGRESS: at 5.36% examples, 478814 words/s, in_qsize 4, out_qsize 0
2020-06-01 18:18:59,786 : INFO : EPOCH 1 - PROGRESS: at 5.73% examples, 479522 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:19:00,837 : INFO : EPOCH 1 - PROGRESS: at 5.89% examples, 471620 words/s, in_qsize 7, out_qsize 1
2020-06-01 18:19:01,853 : INFO : EPOCH 1 - PROGRESS: at 6.19% examples, 469138 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:19:02,883 : INFO : EPOCH 1 - PROGRESS: at 6.51% examples, 469047 words/s, in_qsize 8, out_

2020-06-01 18:20:08,927 : INFO : EPOCH 1 - PROGRESS: at 26.57% examples, 404825 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:20:09,945 : INFO : EPOCH 1 - PROGRESS: at 27.04% examples, 405785 words/s, in_qsize 1, out_qsize 0
2020-06-01 18:20:10,956 : INFO : EPOCH 1 - PROGRESS: at 27.41% examples, 406428 words/s, in_qsize 5, out_qsize 2
2020-06-01 18:20:11,957 : INFO : EPOCH 1 - PROGRESS: at 27.86% examples, 407670 words/s, in_qsize 4, out_qsize 0
2020-06-01 18:20:12,965 : INFO : EPOCH 1 - PROGRESS: at 28.31% examples, 408535 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:20:13,974 : INFO : EPOCH 1 - PROGRESS: at 28.83% examples, 409565 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:20:14,999 : INFO : EPOCH 1 - PROGRESS: at 29.39% examples, 410760 words/s, in_qsize 7, out_qsize 2
2020-06-01 18:20:16,002 : INFO : EPOCH 1 - PROGRESS: at 29.87% examples, 411610 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:20:17,014 : INFO : EPOCH 1 - PROGRESS: at 30.34% examples, 412668 words/s, in_qsiz

2020-06-01 18:21:22,963 : INFO : EPOCH 1 - PROGRESS: at 82.66% examples, 452495 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:21:23,966 : INFO : EPOCH 1 - PROGRESS: at 82.97% examples, 451994 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:21:24,988 : INFO : EPOCH 1 - PROGRESS: at 83.28% examples, 451704 words/s, in_qsize 7, out_qsize 1
2020-06-01 18:21:25,992 : INFO : EPOCH 1 - PROGRESS: at 83.67% examples, 451609 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:21:27,004 : INFO : EPOCH 1 - PROGRESS: at 83.96% examples, 451241 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:21:28,033 : INFO : EPOCH 1 - PROGRESS: at 84.29% examples, 451195 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:21:29,042 : INFO : EPOCH 1 - PROGRESS: at 84.71% examples, 451241 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:21:30,073 : INFO : EPOCH 1 - PROGRESS: at 85.19% examples, 451237 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:21:31,092 : INFO : EPOCH 1 - PROGRESS: at 85.66% examples, 450895 words/s, in_qsiz

2020-06-01 18:22:33,603 : INFO : EPOCH 2 - PROGRESS: at 8.10% examples, 510383 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:22:34,605 : INFO : EPOCH 2 - PROGRESS: at 8.41% examples, 508732 words/s, in_qsize 6, out_qsize 1
2020-06-01 18:22:35,649 : INFO : EPOCH 2 - PROGRESS: at 8.70% examples, 506065 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:22:36,672 : INFO : EPOCH 2 - PROGRESS: at 9.12% examples, 504675 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:22:37,681 : INFO : EPOCH 2 - PROGRESS: at 9.50% examples, 503972 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:22:38,707 : INFO : EPOCH 2 - PROGRESS: at 9.80% examples, 501395 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:22:39,726 : INFO : EPOCH 2 - PROGRESS: at 10.09% examples, 499666 words/s, in_qsize 7, out_qsize 1
2020-06-01 18:22:40,783 : INFO : EPOCH 2 - PROGRESS: at 10.43% examples, 498995 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:22:41,795 : INFO : EPOCH 2 - PROGRESS: at 10.77% examples, 498393 words/s, in_qsize 8, o

...

2020-06-01 18:28:35,642 : INFO : EPOCH 4 - PROGRESS: at 6.82% examples, 516688 words/s, in_qsize 3, out_qsize 1
2020-06-01 18:28:36,639 : INFO : EPOCH 4 - PROGRESS: at 7.14% examples, 514156 words/s, in_qsize 6, out_qsize 1
2020-06-01 18:28:37,672 : INFO : EPOCH 4 - PROGRESS: at 7.41% examples, 511853 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:28:38,708 : INFO : EPOCH 4 - PROGRESS: at 7.71% examples, 509924 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:28:39,729 : INFO : EPOCH 4 - PROGRESS: at 8.05% examples, 508208 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:28:40,742 : INFO : EPOCH 4 - PROGRESS: at 8.42% examples, 509276 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:28:41,745 : INFO : EPOCH 4 - PROGRESS: at 8.83% examples, 510286 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:28:42,749 : INFO : EPOCH 4 - PROGRESS: at 9.25% examples, 510048 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:28:43,758 : INFO : EPOCH 4 - PROGRESS: at 9.66% examples, 511259 words/s, in_qsize 5, out_

2020-06-01 18:29:49,747 : INFO : EPOCH 4 - PROGRESS: at 37.71% examples, 495293 words/s, in_qsize 1, out_qsize 0
2020-06-01 18:29:50,761 : INFO : EPOCH 4 - PROGRESS: at 40.52% examples, 497111 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:29:51,778 : INFO : EPOCH 4 - PROGRESS: at 43.29% examples, 498615 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:29:52,788 : INFO : EPOCH 4 - PROGRESS: at 46.35% examples, 500574 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:29:53,791 : INFO : EPOCH 4 - PROGRESS: at 49.25% examples, 502253 words/s, in_qsize 5, out_qsize 1
2020-06-01 18:29:54,796 : INFO : EPOCH 4 - PROGRESS: at 52.04% examples, 503322 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:29:55,817 : INFO : EPOCH 4 - PROGRESS: at 54.45% examples, 503848 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:29:56,818 : INFO : EPOCH 4 - PROGRESS: at 57.82% examples, 505187 words/s, in_qsize 3, out_qsize 0
2020-06-01 18:29:57,821 : INFO : EPOCH 4 - PROGRESS: at 61.05% examples, 506196 words/s, in_qsiz

2020-06-01 18:31:03,721 : INFO : EPOCH 4 - PROGRESS: at 94.62% examples, 502586 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:31:04,745 : INFO : EPOCH 4 - PROGRESS: at 94.85% examples, 502323 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:31:05,790 : INFO : EPOCH 4 - PROGRESS: at 95.25% examples, 501845 words/s, in_qsize 8, out_qsize 0
2020-06-01 18:31:06,792 : INFO : EPOCH 4 - PROGRESS: at 95.78% examples, 501715 words/s, in_qsize 4, out_qsize 1
2020-06-01 18:31:07,793 : INFO : EPOCH 4 - PROGRESS: at 96.37% examples, 501618 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:31:08,802 : INFO : EPOCH 4 - PROGRESS: at 96.99% examples, 501478 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:31:09,817 : INFO : EPOCH 4 - PROGRESS: at 97.48% examples, 500971 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:31:10,822 : INFO : EPOCH 4 - PROGRESS: at 98.01% examples, 500950 words/s, in_qsize 3, out_qsize 0
2020-06-01 18:31:11,825 : INFO : EPOCH 4 - PROGRESS: at 98.54% examples, 500928 words/s, in_qsiz

2020-06-01 18:32:14,545 : INFO : EPOCH 5 - PROGRESS: at 17.45% examples, 486472 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:32:15,561 : INFO : EPOCH 5 - PROGRESS: at 17.83% examples, 486708 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:32:16,564 : INFO : EPOCH 5 - PROGRESS: at 18.25% examples, 486697 words/s, in_qsize 7, out_qsize 1
2020-06-01 18:32:17,574 : INFO : EPOCH 5 - PROGRESS: at 18.53% examples, 484628 words/s, in_qsize 6, out_qsize 0
2020-06-01 18:32:18,574 : INFO : EPOCH 5 - PROGRESS: at 18.92% examples, 484313 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:32:19,592 : INFO : EPOCH 5 - PROGRESS: at 19.32% examples, 485110 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:32:20,602 : INFO : EPOCH 5 - PROGRESS: at 19.75% examples, 485332 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:32:21,613 : INFO : EPOCH 5 - PROGRESS: at 20.15% examples, 485432 words/s, in_qsize 7, out_qsize 0
2020-06-01 18:32:22,623 : INFO : EPOCH 5 - PROGRESS: at 20.59% examples, 484883 words/s, in_qsiz

2020-06-01 18:33:28,342 : INFO : EPOCH 5 - PROGRESS: at 74.91% examples, 515154 words/s, in_qsize 4, out_qsize 0
2020-06-01 18:33:29,355 : INFO : EPOCH 5 - PROGRESS: at 75.46% examples, 515574 words/s, in_qsize 0, out_qsize 0
2020-06-01 18:33:30,360 : INFO : EPOCH 5 - PROGRESS: at 76.00% examples, 515981 words/s, in_qsize 4, out_qsize 0
2020-06-01 18:33:31,372 : INFO : EPOCH 5 - PROGRESS: at 76.57% examples, 516415 words/s, in_qsize 4, out_qsize 0
2020-06-01 18:33:32,387 : INFO : EPOCH 5 - PROGRESS: at 77.13% examples, 516823 words/s, in_qsize 3, out_qsize 0
2020-06-01 18:33:33,407 : INFO : EPOCH 5 - PROGRESS: at 77.66% examples, 517182 words/s, in_qsize 3, out_qsize 0
2020-06-01 18:33:34,408 : INFO : EPOCH 5 - PROGRESS: at 78.19% examples, 517587 words/s, in_qsize 2, out_qsize 0
2020-06-01 18:33:35,433 : INFO : EPOCH 5 - PROGRESS: at 78.53% examples, 517950 words/s, in_qsize 5, out_qsize 0
2020-06-01 18:33:36,444 : INFO : EPOCH 5 - PROGRESS: at 79.03% examples, 518320 words/s, in_qsiz

In [26]:
#Cek model
namaFileModel = "wiki_word2vec_model"
model = gensim.models.Word2Vec.load(namaFileModel)
hasil = model.most_similar("tdk")
print("tdk:{}".format(hasil))

2020-06-01 18:35:52,233 : INFO : loading Word2Vec object from wiki_word2vec_model
2020-06-01 18:35:53,962 : INFO : loading wv recursively from wiki_word2vec_model.wv.* with mmap=None
2020-06-01 18:35:53,964 : INFO : loading vectors from wiki_word2vec_model.wv.vectors.npy with mmap=None
2020-06-01 18:35:54,323 : INFO : setting ignored attribute vectors_norm to None
2020-06-01 18:35:54,326 : INFO : loading vocabulary recursively from wiki_word2vec_model.vocabulary.* with mmap=None
2020-06-01 18:35:54,327 : INFO : loading trainables recursively from wiki_word2vec_model.trainables.* with mmap=None
2020-06-01 18:35:54,329 : INFO : loading syn1neg from wiki_word2vec_model.trainables.syn1neg.npy with mmap=None
2020-06-01 18:35:54,779 : INFO : setting ignored attribute cum_table to None
2020-06-01 18:35:54,781 : INFO : loaded wiki_word2vec_model
C:\Users\RuiHan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 

tdk:[('yonath', 0.5434169769287109), ('hegerberg', 0.5395534038543701), ('kalaupun', 0.5380741953849792), ('kalanya', 0.535808801651001), ('velipojë', 0.519893229007721), ('ciganlija', 0.5097905993461609), ('idak', 0.5093910694122314), ('yardeni', 0.49315208196640015), ('međica', 0.4843572974205017), ('mengada', 0.48064276576042175)]


In [27]:
model.wv.most_similar("jgn")

[('tacangang', 0.6324961185455322),
 ('doiku', 0.5909720659255981),
 ('kawinin', 0.5854042768478394),
 ('goblok', 0.5775140523910522),
 ('lelalelalela', 0.5674261450767517),
 ('kenak', 0.5658538341522217),
 ('anjeunna', 0.5625079870223999),
 ('ganko', 0.5598341226577759),
 ('koder', 0.5587683916091919),
 ('juo', 0.5562487840652466)]

In [28]:
slang_corpus = pd.read_csv("colloquial-indonesian-lexicon.csv")
slang_corpus

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,"Selamat ulang tahun kakak tulus semoga panjang umur kakak,sehat selalu juga,murah rezeki ya kakak dan jangan sombong kalo sukses di karirnya aminn @tulusm",elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @tulusm,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @tulusm,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
...,...,...,...,...,...,...,...
15001,gataunya,enggak taunya,0,Ini kaya nenek2 ya beb gataunya agnezz @yugimellasih,akronim,0,0
15002,gtau,enggak tau,0,"Stidaknya mrka may berkarya Dan berusaha yg trbaikk bkan kmauan mrka tpi mrka punn mengikutii atassan mrka,so klauu gtau jngnn so ngjudge orng kya gtu,kya sendirinyya punya frestasiii ajjh @meilisa_kyungsoo_",akronim,abreviasi,0
15003,gatau,enggak tau,0,Ih gatau malu,akronim,0,0
15004,fans2,fan-fan,0,Jkt48 adalah tempat di mana sesama fans saling menghujat..men judge sesama fans.. dimana petinggi2 fans club slalu memandang sebelah mata para wota2 lain..yg hanya ingin membuktikan bahwa diri nya adalah yg terbaik di antara fans2 yg lain.. itu alasan knpa aku lebih memilih jalur indefendet ketimbang mengikuti fans club jkt48. Karena mencintai dan mendukung jkt48 gak harus melalui fans club kan ??? Karena fansclub hanya tempat2 orang yg kyk taik.. yg slalu menghujat sesama fans jkt48.. fuck to wota .!!! I love jkt48 .!!!!! wota anak2 sampah..!!!,reduplikasi,naturalisasi,0


In [29]:
slang_corpus = slang_corpus[slang_corpus.columns[0:2]]
slang_corpus

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa
...,...,...
15001,gataunya,enggak taunya
15002,gtau,enggak tau
15003,gatau,enggak tau
15004,fans2,fan-fan


In [39]:
import nltk

for i in tqdm(range(len(df['comment']))):
    for j in range(len(df['comment'][i])):
        try:
            words = df['comment'][i][j]
            corr_comment = []
            if sub_comment in model.wv.vocab:
                sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
                if len(sim_slang) != 0:
                    mode_word = sim_slang.formal.mode()[0]
                    corr_word = corr_word + 1
                    df['comment'][i][j]= corr_comment.append(mode_word)
                else:
                    corr_word = corr_word + 1
                    df['comment'][i][j]= corr_comment.append(sub_comment)
            else:
                rat_words = []
                for sub_comment in slang_corpus['slang']:
                    rat_word = nltk.edit_distance(sub_comment, slang)
                    rat_words.append(rat_word)
                min_rat = rat_words.index(min(rat_words))
                value = str(slang_corpus.formal.loc[min_rat])
                corr_words = corr_word + 1
                corr_comment.append(value)
                df['comment'][i][j]= slang_words.append(sub_comment + " = " + value)
        except:
                pass
                print(df['comment'][i][j])

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

wkwk
enggak
sekalian
pak
selawat
ke
dapur
kan
banyak
koruptor
disana
mantab
ini
baru
dakwah
yang
sejati
kalau
ada
tempat
umum
mah
deh
biasa
ini
luar
biasa
salut
buat
gus
miftah
yang
nyinyir
enggak
pernah
lihat
dan
baca
atau
mengetahui
sejarah
dakwah
walisongo
sunan
kali
jaga
semua
ada
cara
demi
kebaikan
asal
jangan
ajak
dengan
cara
berzina
lakukan
dengan
porsi
dan
sikon
yang
tepat


  0%|▏                                                                                | 4/2000 [00:00<01:27, 22.73it/s]

ada
adabnya
sholawat
lebih
baik
ajak
ke
majelis
ilmu
setiap
ulama
punya
jalan
dan
cara
dakwah
masing
jika
kamu
melihat
kemunkaran
lawan
dengan
tanganmu
kekuatanmu
jika
tidak
mampu
lawan
dengan
mulutmu
jika
tidak
mampu
juga
dengan
hatimu
itulah
selemah
lemahnya
iman
itu
sudah
menjadi
kewajiban
para
ulama
kalo
yang
komen
masih
ada
yang
nyinyir
ulama
berarti
otaknya
sudah
disisipin
politik


  0%|▏                                                                                | 6/2000 [00:00<01:42, 19.37it/s]

...

innalilahi
wa
innailaihi
rajiuun
semoga
para
korban
husnul
khotimah
tetapkan
sebagai
bencana
nasional
pak
presiden
sebelum
anda
kehilangan
dukungan
dari
kami
apakah
perlu
penduduk
pulau
lombok
diungsikan
ke
pulau
lain
dulu
turut
berduka
saudaraku
apapun
yang
terjadi
disana
kami
di
pulau
jawa
tetap
berdoa
buat
kalian
i
love
untuk
all


 98%|████████████████████████████████████████████████████████████████████████████▊ | 1969/2000 [01:22<00:01, 26.77it/s]

sebaiknya
pemerintah
segera
menetapkan
sebagai
bencana
nasional
semoga
pemimpin
kita
punya
kepedulian
terhadap
para
korban
meski
saat
ini
beliau
sedang
menonton
pertandingan
wushu
semoga
saudara
saudara
kita
disana
diberi
ketabahan
dan
kekuatan
oleh
tuhan
amien


 99%|████████████████████████████████████████████████████████████████████████████▉ | 1972/2000 [01:22<00:01, 22.30it/s]

semoga
di
beri
ketabahan
saudara
kita
yang
di
lombok
ya
tuhan
jauhkanlah
bencana
dari
negeri
kami
yang
penting
lanjutkan
libas
semua
demi
dua
periode
harusnya
kepala
daerah
dan
wakilnya
tidak
boleh
jadi
jurkam
alias
netral
hahahaha
semuanya
saja
boleh
pak
jaman
apa


In [40]:
df

,comment
0,"[wkwk, enggak, sekalian, pak, selawat, ke, dapur, kan, banyak, koruptor, disana]"
1,"[mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]"
2,"[salut, buat, gus, miftah]"
3,"[yang, nyinyir, enggak, pernah, lihat, dan, baca, atau, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, semua, ada, cara, demi, kebaikan, asal, jangan, ajak, dengan, cara, berzina, lakukan, dengan, porsi, dan, sikon, yang, tepat]"
4,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...
1995,"[pasti, kpk, memilih, machmud, md, orang, yang, bersih, rekam, jejak, nya, bagus, dan, bukan, dari, kalangan, partai]"
1996,"[hahaha, ingat, dulu, waktu, memilih, mentri, libatin, kpk, juga, tapi, rekomendasi, kpk, enggak, dipakai]"
1997,"[enak, ya, bisa, pakai, jasa, kpk]"
1998,"[basi, pak, pencitraan, pakai, kpk, dulu, menterinujung, nya, main, comot, saja, yang, sudah, jadi, tersangka, kpk, kok, malah, menjabat, hadeh, suram, jok, kampanyemu]"


In [41]:
#simpan hasil si df['stem_comment'] ke csv
df['comment'].to_csv('data_hasil.csv', index=False)

C:\Users\RuiHan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [42]:
dffx=pd.read_csv('data_hasil.csv', names=['hasil_perbaikan'])
dffx

,hasil_perbaikan
0,"['wkwk', 'enggak', 'sekalian', 'pak', 'selawat', 'ke', 'dapur', 'kan', 'banyak', 'koruptor', 'disana']"
1,"['mantab', 'ini', 'baru', 'dakwah', 'yang', 'sejati', 'kalau', 'ada', 'tempat', 'umum', 'mah', 'deh', 'biasa', 'ini', 'luar', 'biasa']"
2,"['salut', 'buat', 'gus', 'miftah']"
3,"['yang', 'nyinyir', 'enggak', 'pernah', 'lihat', 'dan', 'baca', 'atau', 'mengetahui', 'sejarah', 'dakwah', 'walisongo', 'sunan', 'kali', 'jaga', 'semua', 'ada', 'cara', 'demi', 'kebaikan', 'asal', 'jangan', 'ajak', 'dengan', 'cara', 'berzina', 'lakukan', 'dengan', 'porsi', 'dan', 'sikon', 'yang', 'tepat']"
4,"['ada', 'adabnya', 'sholawat', 'lebih', 'baik', 'ajak', 'ke', 'majelis', 'ilmu']"
...,...
1995,"['pasti', 'kpk', 'memilih', 'machmud', 'md', 'orang', 'yang', 'bersih', 'rekam', 'jejak', 'nya', 'bagus', 'dan', 'bukan', 'dari', 'kalangan', 'partai']"
1996,"['hahaha', 'ingat', 'dulu', 'waktu', 'memilih', 'mentri', 'libatin', 'kpk', 'juga', 'tapi', 'rekomendasi', 'kpk', 'enggak', 'dipakai']"
1997,"['enak', 'ya', 'bisa', 'pakai', 'jasa', 'kpk']"
1998,"['basi', 'pak', 'pencitraan', 'pakai', 'kpk', 'dulu', 'menterinujung', 'nya', 'main', 'comot', 'saja', 'yang', 'sudah', 'jadi', 'tersangka', 'kpk', 'kok', 'malah', 'menjabat', 'hadeh', 'suram', 'jok', 'kampanyemu']"
